In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
from tqdm.notebook import tqdm
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models

from sklearn.model_selection import GroupKFold

In [2]:
class CFG:
   debug=True
   dataset_dir="../input/ranzcr-clip-catheter-line-classification/"
   models_dir="./models/" if debug else "../input/efficientnet-lightgbm-models/"
   batch_size=8 if debug else 32
   input_shape=(260,260)

   target_cols=['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline',
      'NGT - Incompletely Imaged', 'NGT - Normal', 'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present']
   n_folds=4

In [3]:
test=pd.read_csv(f"{CFG.dataset_dir}train.csv") if CFG.debug else pd.read_csv(f"{CFG.dataset_dir}sample_submission.csv")

In [4]:
with open(f"{CFG.models_dir}effnet_tuned/model.json","rt") as f:
    effnet_json=f.read()
effnet=models.model_from_json(effnet_json)
effnet.load_weights(f"{CFG.models_dir}effnet_tuned/weight.hdf5")

layer_name="global_average_pooling2d"
hidden_layer_model=models.Model(inputs=effnet.input,outputs=effnet.get_layer(layer_name).output)
hidden_layer_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb7_input (InputL [(None, 260, 260, 3)]     0         
_________________________________________________________________
efficientnetb7 (Functional)  (None, 9, 9, 2560)        64097687  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2560)              0         
Total params: 64,097,687
Trainable params: 63,786,960
Non-trainable params: 310,727
_________________________________________________________________


In [5]:
def preprocess(record):
    if CFG.debug:
        file_bytes=tf.io.read_file(f"{CFG.dataset_dir}train/"+record+".jpg") # f"{}"形式ではTensorの型変換が働かない
    else:
        file_bytes=tf.io.read_file(f"{CFG.dataset_dir}test/"+record+".jpg")
    image=tf.io.decode_jpeg(file_bytes,channels=3)
    image=tf.cast(image,tf.float32)
    image=tf.image.resize(image,CFG.input_shape)
    image/=255
    return image

def decode_string(tensor):
    return [str_bytes.decode() for str_bytes in tensor.numpy()]


dset=tf.data.Dataset.from_tensor_slices(test["StudyInstanceUID"])
dset=dset.map(preprocess,num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(CFG.batch_size)

uid_dset=tf.data.Dataset.from_tensor_slices(test["StudyInstanceUID"]).batch(CFG.batch_size)

In [7]:
ndarray_dict={}
for imgs,uids in tqdm(zip(dset,uid_dset)):
    preds=hidden_layer_model.predict(imgs)
    uids_decoded=decode_string(uids)
    
    for uid,pred in zip(uids_decoded,preds):
        ndarray_dict[uid]=pred
    if CFG.debug:
        break

0it [00:00, ?it/s]

In [8]:
if CFG.debug:
    features_list=[ndarray_dict[uid] for uid in ndarray_dict.keys()]
else:
    features_list=[ndarray_dict[uid] for uid in tqdm(test["StudyInstanceUID"])]
features=np.array(features_list)

In [9]:
scaler=pickle.load(open(f"{CFG.models_dir}minmaxscaler_effnet_tuned.pickle","rb"))
X=scaler.transform(features)

In [10]:
autoencoder_dir=f"{CFG.models_dir}autoencoder_tuned/"
with open(f"{autoencoder_dir}model.json","rt") as f:
    model_json=f.read()
autoencoder=models.model_from_json(model_json)
autoencoder.load_weights(f"{autoencoder_dir}ckpt")
autoencoder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1800)              4609800   
_________________________________________________________________
dense_1 (Dense)              (None, 100)               180100    
_________________________________________________________________
dense_2 (Dense)              (None, 1800)              181800    
_________________________________________________________________
dropout (Dropout)            (None, 1800)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 2560)              4610560   
Total params: 9,582,260
Trainable params: 9,582,260
Non-trainable params: 0
_________________________________________________________________


In [11]:
layer_name="dense_1"
compressing_model=models.Model(inputs=autoencoder.input,outputs=autoencoder.get_layer(layer_name).output)
ae_pred=compressing_model.predict(X)
display(ae_pred.shape)

(8, 100)

In [15]:
def create_model(fold:int):
    dense_dir=f"{CFG.models_dir}effnet_tuned_dense/"
    with open(f"{dense_dir}model.json","rt") as f:
        model_json=f.read()
    dense_model=models.model_from_json(model_json)
    dense_model.load_weights(f"{dense_dir}ckpt_{fold}")
    return dense_model

dense_model_list=[create_model(n) for n in range(CFG.n_folds)]

In [26]:
dense_pred_list=[pd.DataFrame(dense_model_list[n].predict(ae_pred),columns=CFG.target_cols) for n in range(CFG.n_folds)]

submission=test.copy()

for col_name in CFG.target_cols:
    pred_col=pd.DataFrame()
    for n,pred in enumerate(dense_pred_list):
        pred_col[f"fold_{n}"]=pred[col_name]
    submission[col_name]=pred_col.mean(axis=1)
display(submission)

if not CFG.debug:
    submission.to_csv("submission.csv",index=False)

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,0.002587,0.005288,0.009442,0.036597,0.124113,0.078769,0.791030,0.178900,0.440026,0.222484,0.000828,ec89415d1
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,0.002359,0.032266,0.942358,0.056138,0.008361,0.881001,0.014522,0.030994,0.179289,0.936635,0.001491,bf4c6da3c
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,0.000620,0.000599,0.000556,0.001261,0.001752,0.000789,0.001355,0.059175,0.372214,0.632515,0.000285,3fc1c97e5
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0.001556,0.010968,0.006626,0.003903,0.001638,0.010226,0.004772,0.546552,0.286116,0.252574,0.000349,c31019814
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,0.001471,0.002704,0.009839,0.014812,0.021446,0.028405,0.012212,0.008020,0.099802,0.937568,0.000763,207685cd1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30078,1.2.826.0.1.3680043.8.498.74257566841157531124...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5b5b9ac30
30079,1.2.826.0.1.3680043.8.498.46510939987173529969...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7192404d8
30080,1.2.826.0.1.3680043.8.498.43173270582850645437...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d4d1b066d
30081,1.2.826.0.1.3680043.8.498.95092491950130838685...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01a6602b8
